# Modify the genome

Inatall the biopython if you haven't installed.

In [3]:
!pip install biopython


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 28.3 MB/s eta 0:00:00


This step is optional. If using colab, you need to mount your drive so that you can use the data from Google Drive.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from Bio import SeqIO
import pandas as pd

Here we read the "sampled_chr2.vcf" file.This file contains all possible variants in chromosome 2. If you want to modify chromosome 1, then load "sampled_chr1.vcf". After load the file, only keep useful columns.

In [8]:
pd.set_option('display.max_columns', None)
vcf_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/IFN646/sampled_chr2.vcf", comment='#', delimiter='\t', header=None)
vcf_data = vcf_data.drop([0,2,5,6,7,8],axis=1)
new_column_names = ['POS', 'REF', 'ALT', 'HG00142', 'HG00178', 'HG00237', 'HG01521', 'HG03129', 'NA19175', 'NA19462']
vcf_data.columns = new_column_names


In [9]:
vcf_data

,POS,REF,ALT,HG00142,HG00178,HG00237,HG01521,HG03129,NA19175,NA19462
0,10103,C,CCCAGAT,0|0,1|0,1|0,1|1,1|0,1|0,0|1
1,10587,C,G,0|0,0|0,0|0,0|0,0|0,0|0,0|1
2,10596,G,T,0|0,0|0,0|0,0|0,0|0,0|0,0|1
3,10655,C,T,0|0,0|0,0|0,0|0,0|0,0|0,0|1
4,11274,C,G,0|0,0|0,0|0,0|0,0|0,0|0,0|1
...,...,...,...,...,...,...,...,...,...,...
749025,242158047,G,A,0|0,0|0,0|0,0|0,1|0,0|0,0|1
749026,242158137,CT,C,0|0,0|0,0|0,0|0,1|1,0|1,0|1
749027,242163361,G,A,0|0,0|0,0|1,0|1,0|0,0|0,0|0
749028,242171531,C,T,0|0,0|0,0|0,0|0,0|1,0|0,0|0


Load the chromosome 1 genome and create two copies for each person. Due to the limitations of computing resources, I suggest that only one person be processed at a time. Just replace the samples = ["HG00178"] to another sample.

In [10]:
print(vcf_data.head())

fasta_file = '/content/drive/MyDrive/Colab Notebooks/IFN646/chr2.fasta'
#['HG00142', 'HG00178', 'HG00237', 'HG01521', 'HG03129', 'NA19175', 'NA19462']
samples = ["HG00178"]
for record in SeqIO.parse(fasta_file, "fasta"):
    seq_region = list(record.seq)
    samples_seq_1 = {sample: seq_region[:] for sample in samples}
    samples_seq_2 = {sample: seq_region[:] for sample in samples}

     POS REF      ALT HG00142 HG00178 HG00237 HG01521 HG03129 NA19175 NA19462
0  10103   C  CCCAGAT     0|0     1|0     1|0     1|1     1|0     1|0     0|1
1  10587   C        G     0|0     0|0     0|0     0|0     0|0     0|0     0|1
2  10596   G        T     0|0     0|0     0|0     0|0     0|0     0|0     0|1
3  10655   C        T     0|0     0|0     0|0     0|0     0|0     0|0     0|1
4  11274   C        G     0|0     0|0     0|0     0|0     0|0     0|0     0|1


In [ ]:
from tqdm import tqdm
import pdb;


The main function doing the modification. Go through the vcf file line by line. Base on "gt" to decide which copy need to be changed.

In [ ]:
# Initialize offsets for each sample for sequences 1 and 2
offsets_1 = {sample: 0 for sample in samples}
offsets_2 = {sample: 0 for sample in samples}


# Iterate through each variant in the VCF data
for _, var in tqdm(vcf_data.iterrows(), total=vcf_data.shape[0]):
    pos = var["POS"] - 1  # VCF positions are 1-based, but Python is 0-based
    ref = var["REF"]
    alt = var["ALT"]

    # Iterate through each sample
    for sample in samples:
        gt = var[sample]  # Get the genotype for the current sample

        # If the genotype is 1|0, update the first sequence
        if gt == "1|0":
            samples_seq_1[sample][pos+offsets_1[sample]:pos+offsets_1[sample]+len(ref)] = list(alt)
            offsets_1[sample] += len(alt) - len(ref)

        # If the genotype is 0|1, update the second sequence
        elif gt == "0|1":
            samples_seq_2[sample][pos+offsets_2[sample]:pos+offsets_2[sample]+len(ref)] = list(alt)
            offsets_2[sample] += len(alt) - len(ref)

        # If the genotype is 1|1, update both sequences
        elif gt == "1|1":
            samples_seq_1[sample][pos+offsets_1[sample]:pos+offsets_1[sample]+len(ref)] = list(alt)
            samples_seq_2[sample][pos+offsets_2[sample]:pos+offsets_2[sample]+len(ref)] = list(alt)
            offsets_1[sample] += len(alt) - len(ref)
            offsets_2[sample] += len(alt) - len(ref)



100%|██████████| 749030/749030 [1:07:28<00:00, 185.02it/s] 


Check if two sequences are the same.

In [ ]:
samples_seq_1 == samples_seq_2

False

Join the list to a sequence and store the sequence with 80 bits each line in the txt file.

In [ ]:
samples_seq_1 = {sample: ''.join(seq) for sample, seq in samples_seq_1.items()}
samples_seq_2 = {sample: ''.join(seq) for sample, seq in samples_seq_2.items()}

In [ ]:

def wrap_sequence(sequence, width=80):
  """Wraps a sequence into lines of specified width."""
  return '\n'.join([sequence[i:i+width] for i in range(0, len(sequence), width)])

for sample, seq in samples_seq_1.items():
  wrapped_seq = wrap_sequence(seq)
  with open(f'/content/drive/MyDrive/Colab Notebooks/IFN646/{sample}_seq1_chr2.txt', 'w') as f:
    f.write(wrapped_seq)

for sample, seq in samples_seq_2.items():
  wrapped_seq = wrap_sequence(seq)
  with open(f'/content/drive/MyDrive/Colab Notebooks/IFN646/{sample}_seq2_chr2.txt', 'w') as f:
    f.write(wrapped_seq)
